**High-Mountain Asia climate analysis based on gridded observations**

**Katmandhu course, October, 2022**

Topography, temperature

**1. Environment**

In [ ]:
import xarray as xr
import pandas as pd
import numpy as np
import calendar as cld
import datetime
import matplotlib.pyplot as plt
import proplot as plot # New plot library (https://proplot.readthedocs.io/en/latest/)
plot.rc['savefig.dpi'] = 300 # 1200 is too big! #https://proplot.readthedocs.io/en/latest/basics.html#Creating-figures
from scipy import stats
import os
from tqdm import tqdm

In [ ]:
# Where are the data files?
path="/Users/mmenegoz/cours/nepal/data/data_M2_climat/" # Put your own path to the data !

In [ ]:
# Domain study: HMA
lon1=60;lon2=110
lat1=25;lat2=45

**2. Open temperature data file (CRU)**

**or any other data file (e.g. IPSL model)!!!**

In [ ]:
# CRU dataset (https://crudata.uea.ac.uk/cru/data/hrg/cru_ts_4.06/cruts.2205201912.v4.06/tmp/cru_ts4.06.1901.2021.tmp.dat.nc.gz)
# https://www.nature.com/articles/s41597-020-0453-3
# Here we download only a part of the file
file='HMA_cru_ts4.06.1901.2021.tmp.dat.nc'
ds = xr.open_dataset(os.path.join(path,file))
ds # To print the header of the dataset

**3. Extracting regional data and computing seasonal means**

In [ ]:
# Extracting the data over HMA
longitudes=slice(lon1,lon2)
latitudes=slice(lat1,lat2)
temp_HMA=ds.sel(lat=latitudes,lon=longitudes)
temp_HMA # To print the header of the HMA dataset

In [ ]:
# define a function to compute seasonal mean
# Taking into account the differet number of days per months
def season_mean(ds, calendar="standard"):
    # Make a DataArray with the number of days in each month, size = len(time)
    month_length = ds.time.dt.days_in_month

    # Calculate the weights by grouping by 'time.season'
    weights = (
        month_length.groupby("time.season") / month_length.groupby("time.season").sum()
    )

    # Test that the sum of the weights for each season is 1.0
    np.testing.assert_allclose(weights.groupby("time.season").sum().values, np.ones(4))

    # Calculate the weighted average
    return (ds * weights).groupby("time.season").sum(dim="time")

**Below, you might need to change the name of the variable depending on the data that you use!**

**Also, be careful to the unit of the variable!**

In [ ]:
# Computing seasonal mean and ordering properly the seasons
seasonal_mean=season_mean(temp_HMA.tmp).sortby(xr.DataArray(['DJF','MAM','JJA', 'SON'],dims=['season']))

In [ ]:
# Levels of temperature for the plot
levels=plot.arange(-32,32,2)

In [ ]:
# Producing the map of the seasonal mean, excluding the borders of the domain.
f, axs = plot.subplots(proj='cyl',ncols=2, nrows=2, share=1, axwidth=5)
for i, ax in enumerate(axs):
    print('i='+str(i))
    print('ax='+str(ax))
    m = ax.pcolormesh(
        temp_HMA.lon,temp_HMA.lat,
        seasonal_mean[i,:,:],
        levels=levels,
        cmap='coolwarm'
    )
    ax.format(title=seasonal_mean.season.data[i],titlesize='20px')
    
f.colorbar(m, label= '°C')

axs.format(
    gridlinewidth=0.1, gridcolor='gray8', gridalpha=0.5, labels=True, 
    coast=True, ocean=False, oceancolor='gray3', borders=True,
    suptitle="Seasonal mean of temperature, CRU observations",
    lonlines=4, latlines=4, abc=False, latlim=[lat1,lat2],lonlim=[lon1,lon2]
)

**4. Trends**

**Choose the period that you want!**

**Be careful to the consistency between the period that you define and the period available in the dataset!**

In [ ]:
# Period over which computing the trends
date1=datetime.date(year=1901,month=1,day=1)
date2=datetime.date(year=2021,month=12,day=31)
dates=pd.date_range(start=date1, end=date2, freq='Y')
dates.year

**Again, below, check the name of the variable!**

In [ ]:
# Extract seasonal timeseries
Temp=temp_HMA.tmp.sel(time=slice(date1,date2))

**4.1 Check the trend and its level of significance in the location of your choice**

In [ ]:
# location study
lon_loc=87; lat_loc=28 # Everest location!!!
#lon_loc2=72; lat_loc2=32

In [ ]:
# Extraction of the data at the point
temp_loc=Temp.sel(lon=lon_loc, lat=lat_loc, method="nearest")
#temp_loc2=Temp.sel(lon=lon_loc2, lat=lat_loc2, method="nearest")

In [ ]:
# Creating an empty array with the seasonal data
seasonal_T_loc=np.empty(shape=(int(temp_loc.shape[0]/12),4), dtype=float)
seasonal_T_loc.shape

In [ ]:
# Seasonal spatial data
seasonal_T_loc [:,0] = temp_loc.where(Temp['time.season'] == 'DJF').groupby('time.year').mean(dim='time')
seasonal_T_loc [:,1] = temp_loc.where(Temp['time.season'] == 'MAM').groupby('time.year').mean(dim='time')
seasonal_T_loc [:,2] = temp_loc.where(Temp['time.season'] == 'JJA').groupby('time.year').mean(dim='time')
seasonal_T_loc [:,3] = temp_loc.where(Temp['time.season'] == 'SON').groupby('time.year').mean(dim='time')

In [ ]:
# Computing trends
slope_T_loc=np.full(seasonal_T_loc.shape[1:4],np.nan)
pvalue_T_loc=np.full(seasonal_T_loc.shape[1:4],np.nan)
intercept_T_loc=np.full(seasonal_T_loc.shape[1:4],np.nan)
for season in range(4):
    linregress_T_loc = stats.linregress(range(seasonal_T_loc.shape[0]-1), seasonal_T_loc[1:,season])
    slope_T_loc[season] = linregress_T_loc.slope
    intercept_T_loc[season] = linregress_T_loc.intercept
    pvalue_T_loc[season] = linregress_T_loc.pvalue

In [ ]:
seasonal_T_loc.shape

In [ ]:
index=np.arange(0,dates.shape[0])
index.shape

In [ ]:
#Plotting temperature timeseries
f, axs = plot.subplots(ncols=2, nrows=2)
for i, ax in enumerate(axs):
    print('i='+str(i))
    print('ax='+str(ax))
    if pvalue_T_loc[i]<0.05:
        signif='solid'
    else:
        signif='dashed'
    m = ax.scatter(dates,seasonal_T_loc[:,i]-np.mean(seasonal_T_loc[:,i]))
    m = ax.plot(dates,intercept_T_loc[i]+slope_T_loc[i]*index-np.mean(seasonal_T_loc[:,i]),linestyle=signif)
    ax.format(title=f"{seasonal_mean.season.data[i]} trend={slope_T_loc[i]*10:.2f}°C.decade-1; pvalue={pvalue_T_loc[i]:.2g}",titlesize='12px')

axs.format(
    suptitle='Seasonal temperature at lon='+str(lon_loc)+' lat='+str(lat_loc),
    xlabel='year',
    ylabel='temperature anomaly'
)

**4.2. HMA trend maps**

In [ ]:
# Getting the total number of years
int(Temp.shape[0]/12)

In [ ]:
# Creating an empty array with the total seasonal data
seasonal_T=np.empty(shape=(int(Temp.shape[0]/12),4,Temp.shape[1],Temp.shape[2]), dtype=float)
seasonal_T.shape

In [ ]:
Temp.where(Temp['time.season'] == 'DJF').groupby('time.year').mean(dim='time').shape

In [ ]:
# Seasonal spatial data
seasonal_T [:,0,:,:] = Temp.where(Temp['time.season'] == 'DJF').groupby('time.year').mean(dim='time')
seasonal_T [:,1,:,:] = Temp.where(Temp['time.season'] == 'MAM').groupby('time.year').mean(dim='time')
seasonal_T [:,2,:,:] = Temp.where(Temp['time.season'] == 'JJA').groupby('time.year').mean(dim='time')
seasonal_T [:,3,:,:] = Temp.where(Temp['time.season'] == 'SON').groupby('time.year').mean(dim='time')

In [ ]:
# Computing trends
slope_T=np.full(seasonal_T.shape[1:4],np.nan)
pvalue_T=np.full(seasonal_T.shape[1:4],np.nan)

for lon in tqdm(range(seasonal_T.shape[3])):
    for lat in range(seasonal_T.shape[2]):
        for season in range(4):
            linregress_T = stats.linregress(range(seasonal_T.shape[0]-1), seasonal_T[1:,season,lat,lon])
            slope_T[season][lat][lon] = linregress_T.slope
            pvalue_T[season][lat][lon] = linregress_T.pvalue

In [ ]:
# Masking non-significative signals
signif=np.where(pvalue_T<0.05,True,False)

**You might want to modify some features of the plot (levels, xlimit, title, etc...)**

In [ ]:
# Levels of temperature for the plot
levels=plot.arange(-0.3,0.3,0.05)
factor=10 # degree per decade

In [ ]:
# Producing the map of the trends, excluding the borders of the domain.
f, axs = plot.subplots(proj='cyl',ncols=2, nrows=2, share=1, axwidth=5)
for i, ax in enumerate(axs):
    print('i='+str(i))
    print('ax='+str(ax))
    m = ax.pcolormesh(
        temp_HMA.lon,temp_HMA.lat,
        slope_T[i,:,:]*factor,
        levels=levels,
        cmap='coolwarm'
    )
    ax.format(title=seasonal_mean.season.data[i],titlesize='20px')
    
    p = ax.contourf(
    temp_HMA.lon,temp_HMA.lat,
    signif[i,:,:],
    hatches=["", "."], alpha=0
    )
    
f.colorbar(m, label= '°C.decade-1')

axs.format(
    gridlinewidth=0.1, gridcolor='gray8', gridalpha=0.5, labels=True, 
    coast=True, ocean=False, oceancolor='gray3', borders=True,
    suptitle="Seasonal trend of temperature, CRU dataset over "+str(date1)+" - "+str(date2),
    lonlines=4, latlines=4, abc=False, latlim=[lat1,lat2],lonlim=[lon1,lon2]
)

**Now, you can invent your own analysis, e.g:**
* Computing trends where you want and over any period.
* Computing trends as a function of the elevation using the topography in the file below.
* Etc...


In [ ]:
# Topography at 0.25° (http://research.jisao.washington.edu/data_sets/elevation/)
# -> 0.25-degree latitude-longitude resolution elevation (TBASE)
#file_topo="elev.0.25-deg.nc"
file_topo="GMTED2010_15n060_0250deg.nc"
ds_topo = xr.open_dataset(os.path.join(path,file_topo))
ds_topo # ds_topo as dataset

In [ ]:
# Extracting the data over HMA
topo_HMA=ds_topo.sel(longitude=slice(lon1,lon2),latitude=slice(lat1,lat2))

In [ ]:
topo_HMA

In [ ]:
# Quick plot
fig, ax = plt.subplots()
m=ax.pcolormesh(topo_HMA.longitude,topo_HMA.latitude,topo_HMA.elevation,shading='auto')
fig.colorbar(m,label= 'm.asl')
ax.set_title('Topography GMTED with a 0.25° resolution')

In [ ]:
# Interpolation of the orography data on the CRU grid
ds_topo_HMA_out=topo_HMA.interp(latitude=temp_HMA.lat,longitude=temp_HMA.lon)

In [ ]:
# Quick plot
fig, ax = plt.subplots()
m=ax.pcolormesh(ds_topo_HMA_out.longitude,ds_topo_HMA_out.latitude,ds_topo_HMA_out.elevation)
fig.colorbar(m,label= 'm.asl')
ax.set_title('Topography GMTED interpolated on a 0.5° resolution')

In [ ]:
# Levels of temperature for the plot
#levels=np.arange(-0.5,0.5,0.05)
factor_trend=10 # degree per decade
bins=100
levels=np.logspace(0,1,25)

In [ ]:
# We keep only the elevation area > limit m.asl
limit=1000
topo_high=ds_topo_HMA_out.elevation.where(ds_topo_HMA_out.elevation > limit)
trends_high=np.zeros(slope_T.shape)
for i in np.arange(4):
    trends_high[i,:,:]=np.where(ds_topo_HMA_out.elevation > limit,slope_T[i,:,:],np.nan)

**You might want to modify some features of the plot (levels, xlimit, title, etc...)**

In [ ]:
#Plotting trends as a function of elevation
f, axs = plot.subplots(ncols=2, nrows=2,share=1)
for i, ax in enumerate(axs):
    print('i='+str(i))
    print('ax='+str(ax))
    m = ax.scatter(trends_high[i,:,:].flatten()*factor_trend,topo_high.values.flatten(),alpha=0.1)
    ax.format(title=seasonal_mean.season.data[i],titlesize='15px',xlim=(-0.05,0.3))

axs.format(
    suptitle='Temperature trend as a function of the topography',
    xlabel='trends (°C.decade-1)',
    ylabel='Elevation (m. asl)',
    titlesize='15px'
)